In [1]:
import pymongo
from pprint import pprint as pp
from bson.objectid import ObjectId
from bson.json_util import loads

In [2]:
client = pymongo.MongoClient('localhost', 27017, username='root', password='mongo')#%%

# Task 1

In [3]:
db = client['people']
col = db['col']

In [4]:
data = [
    {
        '_id': 1,
        'name': 'Yogesh',
        'age': 20,
        'college': 'iitpkd'
    },
    {
        '_id': 2,
        'name': 'P2',
        'age': 19,
        'college': 'iitm'
    },
    {
        '_id': 3,
        'name': 'P3',
        'age': 19,
        'college': 'iitb'
    },
    {
        'name': 'P4',
        'age': 21,
        'college': 'iitm'
    },
    {
        'name': 'P5',
        'age': 22,
        'college': 'iitd'
    }
]

## Insert

In [5]:
db.drop_collection('col')
data = col.insert_many(data)

In [6]:
data = col.insert_one({'name': 'Hello', 'age': 20, 'college': 'iisc'})

## Find

In [7]:
pp([x for x in col.find({'age': {'$lte': 20}})])

[{'_id': 1, 'age': 20, 'college': 'iitpkd', 'name': 'Yogesh'},
 {'_id': 2, 'age': 19, 'college': 'iitm', 'name': 'P2'},
 {'_id': 3, 'age': 19, 'college': 'iitb', 'name': 'P3'},
 {'_id': ObjectId('609947048c7b45c9cdc4d5fa'),
  'age': 20,
  'college': 'iisc',
  'name': 'Hello'}]


In [8]:
pp(col.find_one({'name': 'Yogesh'}))

{'_id': 1, 'age': 20, 'college': 'iitpkd', 'name': 'Yogesh'}


## Update

In [9]:
query = {'name': 'P3'}
new_value = {'$set': {'college': 'iitk'}}
res = col.update_one(query, new_value)
pp(col.find_one(query))


{'_id': 3, 'age': 19, 'college': 'iitk', 'name': 'P3'}


## Replace

In [10]:
query = {'name': 'P5'}
res = col.replace_one(query, {'name': 'P6', 'age': 21, 'college': 'iisc'})

In [11]:
list(col.find({}))

[{'_id': 1, 'name': 'Yogesh', 'age': 20, 'college': 'iitpkd'},
 {'_id': 2, 'name': 'P2', 'age': 19, 'college': 'iitm'},
 {'_id': 3, 'name': 'P3', 'age': 19, 'college': 'iitk'},
 {'_id': ObjectId('609947048c7b45c9cdc4d5f8'),
  'name': 'P4',
  'age': 21,
  'college': 'iitm'},
 {'_id': ObjectId('609947048c7b45c9cdc4d5f9'),
  'name': 'P6',
  'age': 21,
  'college': 'iisc'},
 {'_id': ObjectId('609947048c7b45c9cdc4d5fa'),
  'name': 'Hello',
  'age': 20,
  'college': 'iisc'}]

## Delete

In [12]:
query = {'name': {'$regex': r'\w*4|6$'}}
res = col.delete_many(query)

## Sort

In [13]:
list(col.find({}).sort('age', -1))

[{'_id': 1, 'name': 'Yogesh', 'age': 20, 'college': 'iitpkd'},
 {'_id': ObjectId('609947048c7b45c9cdc4d5fa'),
  'name': 'Hello',
  'age': 20,
  'college': 'iisc'},
 {'_id': 2, 'name': 'P2', 'age': 19, 'college': 'iitm'},
 {'_id': 3, 'name': 'P3', 'age': 19, 'college': 'iitk'}]

# Task 2

In [3]:
with open('primer-dataset.json') as f:
    data = f.readlines()

In [4]:
res = []
for d in data:
    res.append(loads(d))

data = res

In [5]:
db = client['test_db']
db.drop_collection('restaurant')
col = db['restaurant']

In [6]:
col.insert_many(data)

In [7]:
pp(col.find_one(), indent=2)

{ '_id': ObjectId('60995b2a25c067c12a8fa0b9'),
  'address': { 'building': '1007',
               'coord': [-73.856077, 40.848447],
               'street': 'Morris Park Ave',
               'zipcode': '10462'},
  'borough': 'Bronx',
  'cuisine': 'Bakery',
  'grades': [ { 'date': datetime.datetime(2014, 3, 3, 0, 0),
                'grade': 'A',
                'score': 2},
              { 'date': datetime.datetime(2013, 9, 11, 0, 0),
                'grade': 'A',
                'score': 6},
              { 'date': datetime.datetime(2013, 1, 24, 0, 0),
                'grade': 'A',
                'score': 10},
              { 'date': datetime.datetime(2011, 11, 23, 0, 0),
                'grade': 'A',
                'score': 9},
              { 'date': datetime.datetime(2011, 3, 10, 0, 0),
                'grade': 'B',
                'score': 14}],
  'name': 'Morris Park Bake Shop',
  'restaurant_id': '30075445'}


In [12]:
agg = col.aggregate([
    {
        '$group':
            {
                '_id':
                    {
                        'cuisine': '$cuisine',
                    },
                'count': {'$sum': 1}
            }
    },
    {
        '$match': {'count': {'$gte': 100}}
    },
    {
        '$project':
            {
                'cuisine': '$_id.cuisine',
                'count': 1,
                '_id': 0

            }
    },
    {'$sort': {'cuisine': pymongo.ASCENDING}}
])


In [ ]:
for doc in agg:
    print(doc)

In [13]:
len(list(agg))

35

In [20]:
col.find_one().keys()

dict_keys(['_id', 'address', 'borough', 'cuisine', 'grades', 'name', 'restaurant_id'])

In [21]:
for doc in col.aggregate([
    {
        '$match': {'cuisine': 'Hawaiian'}
    },
    {
        '$unwind': "$grades"
    },
    {
        '$match': {'grades.grade': 'A'}
    },
    {
        '$group':
            {
                '_id': {'cuisine': '$cuisine', 'name': '$name'},
                'count': {'$sum': 1}
            }
    },
    {
        '$project':
            {
                'name': '$_id.name',
                '_id': 0
            }
    },
    {
        '$sort': {'name': pymongo.ASCENDING}
    }
]):
    pp(doc)

{'name': 'General Assembly'}
{'name': 'Makana'}
{'name': 'Onomea'}


In [22]:
for doc in col.aggregate([
    {
        '$group': {
            '_id': {'cuisine': '$cuisine'},
            'count': {'$sum': 1}
        },
    },
    {
        '$project': {
            'cuisine': '$_id.cuisine',
            '_id': 0,
            'count': 1
        }
    },
    {
        '$sort': {'count': pymongo.DESCENDING}
    }
]):
    pp(doc)

{'count': 6183, 'cuisine': 'American'}
{'count': 2418, 'cuisine': 'Chinese'}
{'count': 1214, 'cuisine': 'Café/Coffee/Tea'}
{'count': 1163, 'cuisine': 'Pizza'}
{'count': 1069, 'cuisine': 'Italian'}
{'count': 1011, 'cuisine': 'Other'}
{'count': 850,
 'cuisine': 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)'}
{'count': 760, 'cuisine': 'Japanese'}
{'count': 754, 'cuisine': 'Mexican'}
{'count': 691, 'cuisine': 'Bakery'}
{'count': 657, 'cuisine': 'Caribbean'}
{'count': 637, 'cuisine': 'Spanish'}
{'count': 479, 'cuisine': 'Donuts'}
{'count': 468, 'cuisine': 'Pizza/Italian'}
{'count': 459, 'cuisine': 'Sandwiches'}
{'count': 433, 'cuisine': 'Hamburgers'}
{'count': 410, 'cuisine': 'Chicken'}
{'count': 348, 'cuisine': 'Ice Cream, Gelato, Yogurt, Ices'}
{'count': 344, 'cuisine': 'French'}
{'count': 321, 'cuisine': 'Delicatessen'}
{'count': 316, 'cuisine': 'Indian'}
{'count': 316, 'cuisine': 'Jewish/Kosher'}
{'count': 309, 'cuisine': 'Asian'}
{'count': 285, 'cuisine': 'Thai'}
{'

In [23]:
for doc in col.aggregate([
    {
        '$unwind': '$grades'
    },
    {
        '$group': {
            '_id': {
                'id': '$_id',
                'name': '$name',
            },
            'total score': {'$sum': '$grades.score'}
        }
    },
    {
        '$project': {
            'name': '$_id.name',
            'id': '$_id.id',
            'total score': 1,
            '_id': 0
        }
    },
    {
        '$sort': {'total score': pymongo.DESCENDING}
    },
    {
        '$limit': 5
    }
]):
    pp(doc)

{'id': ObjectId('6099470a8c7b45c9cdc50cc4'),
 'name': 'Red Chopstick',
 'total score': 254}
{'id': ObjectId('6099470a8c7b45c9cdc4ede4'),
 'name': 'Nios Restaurant',
 'total score': 227}
{'id': ObjectId('6099470a8c7b45c9cdc4d686'),
 'name': 'Nanni Restaurant',
 'total score': 225}
{'id': ObjectId('6099470a8c7b45c9cdc5007d'),
 'name': 'Amici 36',
 'total score': 215}
{'id': ObjectId('6099470a8c7b45c9cdc514af'),
 'name': 'Cheikh Umar Futiyu Restaurant',
 'total score': 212}


In [24]:
col.find_one({})

{'_id': ObjectId('6099470a8c7b45c9cdc4d5fb'),
 'address': {'building': '1007',
  'coord': [-73.856077, 40.848447],
  'street': 'Morris Park Ave',
  'zipcode': '10462'},
 'borough': 'Bronx',
 'cuisine': 'Bakery',
 'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
   'grade': 'A',
   'score': 2},
  {'date': datetime.datetime(2013, 9, 11, 0, 0), 'grade': 'A', 'score': 6},
  {'date': datetime.datetime(2013, 1, 24, 0, 0), 'grade': 'A', 'score': 10},
  {'date': datetime.datetime(2011, 11, 23, 0, 0), 'grade': 'A', 'score': 9},
  {'date': datetime.datetime(2011, 3, 10, 0, 0), 'grade': 'B', 'score': 14}],
 'name': 'Morris Park Bake Shop',
 'restaurant_id': '30075445'}

In [30]:
for doc in col.aggregate([
    {
        '$project': {'_id': 1, 'grades': 1}
    },
    {
        '$unwind': '$grades'
    },
    {
        '$group': {
            '_id': '$_id',
            'total grades': {'$sum': 1}
        }
    },
    {'$sort': {'total grades': pymongo.DESCENDING}},
    {'$limit': 4}
]):
    pp(doc)

{'_id': ObjectId('6099470a8c7b45c9cdc4fe6b'), 'total grades': 9}
{'_id': ObjectId('6099470a8c7b45c9cdc4eebe'), 'total grades': 9}
{'_id': ObjectId('6099470a8c7b45c9cdc4f55e'), 'total grades': 9}
{'_id': ObjectId('6099470a8c7b45c9cdc4ee93'), 'total grades': 9}
